# Import Data

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('DailyPrices.csv')
date = df['Date']
prices = df.iloc[:,1:] #drop the first column of date

# Return Calculation

In [3]:
def return_calculation(df):
    pct_change = pd.DataFrame()
    for column in prices.columns:
        pct_change[column] = (df[column].diff()/df[column].shift(1))[1:]
    return pct_change
return_df = return_calculation(prices)
return_df.to_csv('return_df')
return_df

/var/folders/2n/lm0km0mn4zlf79tvv2679h740000gn/T/ipykernel_70379/545654781.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pct_change[column] = (df[column].diff()/df[column].shift(1))[1:]


,SPY,AAPL,MSFT,AMZN,NVDA,GOOGL,TSLA,GOOG,BRK-B,META,...,CI,ETN,SLB,PGR,SCHW,LRCX,ZTS,C,BSX,AMT
1,-0.010544,-0.013611,-0.016667,-0.002425,-0.020808,-0.017223,-0.025076,-0.016915,-0.016854,-0.030479,...,-0.001180,-0.010593,0.033107,-0.010428,-0.019242,-0.004236,-0.015244,0.001846,-0.012198,-0.026355
2,-0.003773,-0.008215,-0.010974,-0.010980,-0.013336,-0.009643,0.015581,-0.011042,-0.003890,-0.011103,...,-0.004641,0.008449,-0.014118,0.000572,0.001848,-0.008019,-0.000892,-0.012695,-0.002717,0.013275
3,0.017965,0.009254,0.019111,0.026723,0.018795,0.024717,0.033817,0.027912,0.016089,0.011669,...,0.016652,0.020295,-0.008030,0.038537,0.018731,0.012279,0.022698,0.008503,0.026994,0.020930
4,0.006536,-0.009618,0.001666,0.002626,0.020126,-0.009776,0.019598,-0.009595,0.008184,0.010412,...,0.002448,0.013945,0.029951,0.015880,0.019083,0.016574,-0.011908,0.026116,0.029901,0.008362
5,0.015535,0.018840,0.022977,0.026575,0.028377,0.020945,0.036023,0.021568,0.008576,0.043749,...,0.007327,0.017244,0.038774,-0.004179,0.018863,0.026460,0.036721,0.015431,0.005385,-0.000306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,0.000586,0.016913,-0.003513,-0.002920,0.001503,0.005895,-0.033201,0.004772,0.006986,0.007459,...,0.007485,0.006938,0.010399,0.013118,-0.006183,0.020125,-0.003329,-0.001639,0.001890,-0.003386
262,-0.002074,0.006181,-0.001246,-0.016788,-0.010144,-0.001230,0.004599,-0.000936,0.000135,0.008329,...,-0.002453,-0.013644,-0.012743,0.013589,-0.002247,-0.016519,0.012970,0.000938,0.000566,-0.012087
263,-0.009193,-0.019992,-0.023977,-0.017002,-0.029435,-0.031150,-0.014672,-0.030541,-0.009879,-0.017701,...,0.009450,-0.006986,-0.010591,0.001544,-0.018361,-0.010062,-0.002748,-0.008903,0.020177,0.000282
264,-0.016528,-0.008889,-0.003866,-0.044053,-0.028931,-0.024675,-0.026239,-0.023999,-0.009651,-0.013148,...,0.012216,-0.018635,-0.016223,-0.002032,-0.011646,-0.013686,-0.026725,-0.013948,-0.002403,-0.045601


# VaR calculation

In [4]:
def calculate_var(returns, alpha=0.05):
    sorted_returns = np.sort(returns)
    index = int(alpha * len(sorted_returns))
    return -sorted_returns[index]

### 1. Normal Distribution Simulation

In [9]:
#normal distribution simulation
df1 = pd.DataFrame()
for column in return_df.columns:
    m = np.mean(return_df[column]) #mean valkue of each stock
    sd = np.std(return_df[column]) # standard deviation of each stock
    sim_normal_return = np.random.normal(m, sd, len(return_df[column]))
    df1[column] = sim_normal_return

#VaR calculation using writen function
var_list = []
for column in df1.columns:
    var_value = calculate_var(df1[column], alpha=0.05)
    var_series = pd.Series([var_value], index=[column])
    var_list.append(var_series)
var_df = pd.concat(var_list, axis=0)
var_df

#VaR calculation using built-in percentile function
var_list = []
for column in df1.columns:
    var_value = - np.percentile(df1[column],5) #5是分位数
    var_series = pd.Series([var_value],index=[column])
    var_list.append(var_series)
var_df_builtin = pd.concat(var_list)
var_df_builtin
    

/var/folders/2n/lm0km0mn4zlf79tvv2679h740000gn/T/ipykernel_70379/1932564048.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[column] = sim_normal_return


SPY     0.018677
AAPL    0.027738
MSFT    0.029943
AMZN    0.038534
NVDA    0.051306
          ...   
LRCX    0.045275
ZTS     0.027790
C       0.027801
BSX     0.022382
AMT     0.027916
Length: 101, dtype: float64

### 2. Historical VaR

In [53]:
l2 = []
for column in return_df.columns:
    var = calculate_var(return_df[column], alpha = 0.05)    
    series = pd.Series([var], index=[column])
    l2.append(series)
var_df2 = pd.concat(l2)
var_df2

SPY     0.016528
AAPL    0.030039
MSFT    0.025767
AMZN    0.039792
NVDA    0.040851
          ...   
LRCX    0.040174
ZTS     0.024978
C       0.028694
BSX     0.018148
AMT     0.031223
Length: 101, dtype: float64

### 3. Exponentially Weighted VaR

In [64]:
#give exponential weighted data
def ewCovar(x, lam):
    m, n = x.shape
    w = np.array([(1-lam)*lam**(m-i) for i in range(m)])
    w /= np.sum(w)
    xm = np.mean(x, axis=0)
    x -= xm
    return np.cov(x, aweights=w, rowvar=False)
w_cov = ewCovar(return_df, 0.74)


101

### 4. Simulate T distribution

### 5. AR simulation 

In [52]:
# Example: AR(1) model fitting and simulation
model = ARIMA(meta, order=(1,0,0))
model_fit = model.fit()
print(model_fit.summary())

innovations = np.random.normal(0, 1, 1000)
ar_simulated = ar1_simulation(meta, model_fit.params, innovations)

# VaR calculation for simulated data
vaR_ar1 = calculate_var(ar_simulated)
print(f"AR(1) VaR: {vaR_ar1}")


NameError: name 'ARIMA' is not defined

# Expected Shortfalls